# Implementation of TextRank


The input text is given below

In [20]:
Text = "Remind me to go to california on the 6th of june along with Michael for attending the international conference on social studies."

### Cleaning Text Data

The raw input text is cleaned off non-printable characters (if any) and turned into lower case.
The processed input text is then tokenized using NLTK library functions. 

In [19]:

import nltk
from nltk import word_tokenize
import string

nltk.download('punkt')

def clean(text):
    text = text.lower()
    printable = set(string.printable)
    #text = (lambda x: x in printable, text) #filter funny characters, if any.
    return text

Cleaned_text = clean(Text)

text = word_tokenize(str(Cleaned_text))

print (text)

['remind', 'me', 'to', 'go', 'to', 'california', 'on', 'the', '6th', 'of', 'june', 'along', 'with', 'michael', 'for', 'attending', 'the', 'international', 'conference', 'on', 'social', 'studies', '.']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\assas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### POS Tagging For Lemmatization

NLTK is again used for <b>POS tagging</b> the input text so that the words can be lemmatized based on their POS tags.

Description of POS tags: 


http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [3]:
nltk.download('averaged_perceptron_tagger')
  
POS_tag = nltk.pos_tag(text)

print ("Tokenized Text with POS tags: \n")
print (POS_tag)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\assas\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Tokenized Text with POS tags: 

[('remind', 'VB'), ('me', 'PRP'), ('to', 'TO'), ('go', 'VB'), ('to', 'TO'), ('california', 'VB'), ('on', 'IN'), ('the', 'DT'), ('6th', 'CD'), ('of', 'IN'), ('june', 'NN'), ('along', 'IN'), ('with', 'IN'), ('michael', 'NN'), ('for', 'IN'), ('attending', 'VBG'), ('the', 'DT'), ('international', 'JJ'), ('conference', 'NN'), ('on', 'IN'), ('social', 'JJ'), ('studies', 'NNS'), ('.', '.')]


### Lemmatization

The tokenized text (mainly the nouns and adjectives) is normalized by <b>lemmatization</b>.
In lemmatization different grammatical counterparts of a word will be replaced by single
basic lemma. For example, 'glasses' may be replaced by 'glass'. 

Details about lemmatization: 
    
https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html

In [4]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

adjective_tags = ['JJ','JJR','JJS']

lemmatized_text = []

for word in POS_tag:
    if word[1] in adjective_tags:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
    else:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
        
print ("Text tokens after lemmatization of adjectives and nouns: \n")
print (lemmatized_text)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\assas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Text tokens after lemmatization of adjectives and nouns: 

['remind', 'me', 'to', 'go', 'to', 'california', 'on', 'the', '6th', 'of', 'june', 'along', 'with', 'michael', 'for', 'attending', 'the', 'international', 'conference', 'on', 'social', 'study', '.']


### POS tagging for Filtering

The <b>lemmatized text</b> is <b>POS tagged</b> here. The tags will be used for filtering later on.

In [5]:
POS_tag = nltk.pos_tag(lemmatized_text)

print ("Lemmatized text with POS tags: \n")
print (POS_tag)

Lemmatized text with POS tags: 

[('remind', 'VB'), ('me', 'PRP'), ('to', 'TO'), ('go', 'VB'), ('to', 'TO'), ('california', 'VB'), ('on', 'IN'), ('the', 'DT'), ('6th', 'CD'), ('of', 'IN'), ('june', 'NN'), ('along', 'IN'), ('with', 'IN'), ('michael', 'NN'), ('for', 'IN'), ('attending', 'VBG'), ('the', 'DT'), ('international', 'JJ'), ('conference', 'NN'), ('on', 'IN'), ('social', 'JJ'), ('study', 'NN'), ('.', '.')]


## POS Based Filtering

Any word from the lemmatized text, which isn't a noun, adjective, or gerund (or a 'foreign word'), is here
considered as a <b>stopword</b> (non-content). This is based on the assumption that usually keywords are noun,
adjectives or gerunds. 

Punctuations are added to the stopword list too.

In [6]:
stopwords = []

wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VBG','FW'] 

for word in POS_tag:
    if word[1] not in wanted_POS:
        stopwords.append(word[0])

punctuations = list(str(string.punctuation))

stopwords = stopwords + punctuations

In [7]:
stopword_file = open("long_stopwords.txt", "r")
#Source = https://www.ranks.nl/stopwords

lots_of_stopwords = []

for line in stopword_file.readlines():
    lots_of_stopwords.append(str(line.strip()))

stopwords_plus = []
stopwords_plus = stopwords + lots_of_stopwords
stopwords_plus = set(stopwords_plus)

#Stopwords_plus contain total set of all stopwords

### Removing Stopwords 

Removing stopwords from lemmatized_text. 
Processeced_text condtains the result.

In [8]:
processed_text = []
for word in lemmatized_text:
    if word not in stopwords_plus:
        processed_text.append(word)
print (processed_text)

['june', 'michael', 'attending', 'international', 'conference', 'social', 'study']


## Vocabulary Creation

Vocabulary will only contain unique words from processed_text.

In [9]:
vocabulary = list(set(processed_text))

print (vocabulary)

['attending', 'conference', 'michael', 'study', 'social', 'international', 'june']


### Building Graph

TextRank is a graph based model, and thus it requires us to build a graph. Each words in the vocabulary will serve as a vertex for graph. The words will be represented in the vertices by their index in vocabulary list.  

The weighted_edge matrix contains the information of edge connections among all vertices.
I am building wieghted undirected edges.

weighted_edge[i][j] contains the weight of the connecting edge between the word vertex represented by vocabulary index i and the word vertex represented by vocabulary j.

If weighted_edge[i][j] is zero, it means no edge connection is present between the words represented by index i and j.

There is a connection between the words (and thus between i and j which represents them) if the words co-occur within a window of a specified 'window_size' in the processed_text.

The value of the weighted_edge[i][j] is increased by (1/(distance between positions of words currently represented by i and j)) for every connection discovered between the same words in different locations of the text. 

The covered_coocurrences list (which is contain the list of pairs of absolute positions in processed_text of the words whose coocurrence at that location is already checked) is managed so that the same two words located in the same positions in processed_text are not repetitively counted while sliding the window one text unit at a time.

The score of all vertices are intialized to one. 

Self-connections are not considered, so weighted_edge[i][i] will be zero.

In [10]:
import numpy as np
import math
vocab_len = len(vocabulary)

weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

score = np.zeros((vocab_len),dtype=np.float32)
window_size = 3
covered_coocurrences = []

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size)):
                
                window_end = window_start+window_size
                
                window = processed_text[window_start:window_end]
                
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])


### Calculating weighted summation of connections of a vertex

inout[i] will contain the sum of all the undirected connections\edges associated withe the vertex represented by i.

In [11]:
inout = np.zeros((vocab_len),dtype=np.float32)

for i in range(0,vocab_len):
    for j in range(0,vocab_len):
        inout[i]+=weighted_edge[i][j]

### Scoring Vertices

The formula used for scoring a vertex represented by i is:

score[i] = (1-d) + d x [ Summation(j) ( (weighted_edge[i][j]/inout[j]) x score[j] ) ] where j belongs to the list of vertieces that has a connection with i. 

d is the damping factor.

The score is iteratively updated until convergence. 

In [12]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(0,vocab_len):
        
        summation = 0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i] = (1-d) + d*(summation)
    
    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print ("Converging at iteration "+str(iter)+"....")
        break


Converging at iteration 23....


In [13]:
for i in range(0,vocab_len):
    print ("Score of "+vocabulary[i]+": "+str(score[i]))

Score of attending: 1.2422547
Score of conference: 1.0685682
Score of michael: 1.0685663
Score of study: 0.15
Score of social: 0.68930006
Score of international: 1.2422433
Score of june: 0.6892986


### Phrase Partiotioning

Paritioning lemmatized_text into phrases using the stopwords in it as delimeters.
The phrases are also candidates for keyphrases to be extracted. 

In [14]:
phrases = []

phrase = " "
for word in lemmatized_text:
    
    if word in stopwords_plus:
        if phrase!= " ":
            phrases.append(str(phrase).strip().split())
        phrase = " "
    elif word not in stopwords_plus:
        phrase+=str(word)
        phrase+=" "

print ("Partitioned Phrases (Candidate Keyphrases): \n")
print (phrases)

Partitioned Phrases (Candidate Keyphrases): 

[['june'], ['michael'], ['attending'], ['international', 'conference'], ['social', 'study']]


### Create a list of unique phrases.

Repeating phrases\keyphrase candidates has no purpose here, anymore. 

In [15]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)

print ("Unique Phrases (Candidate Keyphrases): \n")
print (unique_phrases)

Unique Phrases (Candidate Keyphrases): 

[['june'], ['michael'], ['attending'], ['international', 'conference'], ['social', 'study']]


### Thinning the list of candidate-keyphrases.

Removing single word keyphrases-candidates that are present multi-word alternatives. 

In [16]:
for word in vocabulary:
    #print word
    for phrase in unique_phrases:
        if (word in phrase) and ([word] in unique_phrases) and (len(phrase)>1):
            #if len(phrase)>1 then the current phrase is multi-worded.
            #if the word in vocabulary is present in unique_phrases as a single-word-phrase
            # and at the same time present as a word within a multi-worded phrase,
            # then I will remove the single-word-phrase from the list.
            unique_phrases.remove([word])
            
print ("Thinned Unique Phrases (Candidate Keyphrases): \n")
print (unique_phrases)    

Thinned Unique Phrases (Candidate Keyphrases): 

[['june'], ['michael'], ['attending'], ['international', 'conference'], ['social', 'study']]


### Scoring Keyphrases

Scoring the phrases (candidate keyphrases) and building up a list of keyphrases\keywords
by listing untokenized versions of tokenized phrases\candidate-keyphrases.
Phrases are scored by adding the score of their members (words\text-units that were ranked by the graph algorithm)


In [17]:
phrase_scores = []
keywords = []
for phrase in unique_phrases:
    phrase_score=0
    keyword = ''
    for word in phrase:
        keyword += str(word)
        keyword += " "
        phrase_score+=score[vocabulary.index(word)]
    phrase_scores.append(phrase_score)
    keywords.append(keyword.strip())

i=0
for keyword in keywords:
    print ("Keyword: '"+str(keyword)+"', Score: "+str(phrase_scores[i]))
    i+=1

Keyword: 'june', Score: 0.6892986297607422
Keyword: 'michael', Score: 1.0685663223266602
Keyword: 'attending', Score: 1.2422547340393066
Keyword: 'international conference', Score: 2.3108115196228027
Keyword: 'social study', Score: 0.8393000662326813


### Ranking Keyphrases

Ranking keyphrases based on their calculated scores. Displaying top keywords_num no. of keyphrases.

In [18]:
sorted_index = np.flip(np.argsort(phrase_scores),axis=0)
sorted_index.shape
keywords_num = 5



print ("Keywords:\n")

for i in range(0,keywords_num):
    print (str(keywords[sorted_index[i]])+", ",)

Keywords:

international conference, 
attending, 
michael, 
social study, 
june, 
